In [118]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [119]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import re

In [120]:
df = pd.read_csv('train.csv')

In [121]:
df = df.dropna(subset=['registered_year'])

In [122]:
df.fillna(0, inplace=True)

In [123]:
# KMS __________________________________________________

df['kms_driven'] = df['kms_driven'].str.replace('Kms', '')
df['kms_driven'] = df['kms_driven'].str.replace(',', '').str.strip()

df = df.astype({
    'kms_driven': float,
})
# ENGINE __________________________________________________

df['engine_capacity'] = df['engine_capacity'].str.replace('cc', '')

df = df.astype({
    'engine_capacity': float,
})

# MAX POWER __________________________________________________
#convert to bhp
Correct_Max_Power = []
for power in df['max_power']:
    if pd.notna(power):
        if isinstance(power, str):
            if 'bhp' in power:
                num_bhp = float(re.search(r'\d+.?\d', power).group())
                Correct_Max_Power.append(num_bhp)
            elif 'PS' in power:
                num_ps = float(re.search(r'\d+.?\d', power).group())
                num_bhp = round(num_ps * 0.98632, 2)
                Correct_Max_Power.append(num_bhp)
            else:
                num = re.search(r'\d+\.?\d*', power)
                Correct_Max_Power.append(float(num.group()) if num else None)
        else:
            Correct_Max_Power.append(power)
    else:
        Correct_Max_Power.append(None)


# Convert not NaN to float
Correct_Max_Power = [float(value) if value is not None else value for value in Correct_Max_Power]

df['max_power'] = Correct_Max_Power

df = df.astype({
    'max_power': float,
})
#assign value to blank lines
df['mileage'] = df['mileage'].fillna(0)

df = df[df['mileage'].notna()]
consumo_correto = []

for i in df['mileage']:
    match = re.search(r'(\d+.\d+|\d+)', str(i))  # Encontrar números (com ou sem ponto decimal)

    if match:
        valor = float(match.group())

        if 'km/kg' in str(i):
            valor *= 1.40  # Converter de km/kg para kmpl

        consumo_correto.append(valor)

df['mileage'] = consumo_correto

df = df.astype({
    'mileage': float,
})
df.loc[df["insurance"] == "Third Party insurance","insurance"] = "Third Party"
df.loc[df["insurance"] == "1","insurance"] = "First"
df.loc[df["insurance"] == "2","insurance"] = "Second"
df.loc[df["insurance"] == "Not Available","insurance"] = np.nan
df["insurance"] = df["insurance"].fillna(df["insurance"].mode()[0])

In [124]:
# Transmissao _____________________________________

transmissao = df['full_name']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_nome'] = df['full_name'].map(idf)
df = df.astype({
    'id_nome': float,
})

In [125]:
# Transmissao _____________________________________

transmissao = df['registered_year']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_ano'] = df['registered_year'].map(idf)
df = df.astype({
    'id_ano': float,
})



In [126]:
# Transmissao _____________________________________

transmissao = df['insurance']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_insurance'] = df['insurance'].map(idf)
df = df.astype({
    'id_insurance': float,
})

In [127]:
# Transmissao _____________________________________

transmissao = df['transmission_type']

n_docs = len(df)


transmissao_counts = transmissao.value_counts()

idf = {}

for tipo_transmissao, count in transmissao_counts.items():
    idf[tipo_transmissao] = np.log10(n_docs / (count + 1))  #+1 to avoid division by zero

# Create new column 
df['id_transmission_type'] = df['transmission_type'].map(idf)
df = df.astype({
    'id_transmission_type': float,
})

In [128]:
# Substituir valores na coluna 'owner_type'
df['owner_type'] = df['owner_type'].replace({
    'First Owner': '1.0',
    'Second Owner': '2.0',
    'Third Owner': '3.0',
    'Fourth Owner': '4.0',
    'Fifth Owner': '5.0'
})

# Converter para float
df['owner_type'] = df['owner_type'].astype(float)


In [129]:
#Create ID IDF  

# FUEL TYPE __________________________________________________

fuel_types = df['fuel_type']
n_docs = len(df)
idf = {}

# Uniques
unique_fuel_types = fuel_types.drop_duplicates()

print("FUEL TYPE".center(50, "_"))

for fuel_type in unique_fuel_types:
    k = sum(fuel_type == val for val in fuel_types)  
    idf[fuel_type] = np.log10(n_docs / (k + 1))  #+1 to avoid division by zero
    print(f'{fuel_type:>15}: {idf[fuel_type]:>10}')

# Create new column 
df['id_fuel_type'] = df['fuel_type'].map(idf)

df = df.astype({
    'id_fuel_type': float,
})

____________________FUEL TYPE_____________________
         Petrol: 0.18705300189294208
         Diesel: 0.5001184047638285
            CNG: 1.5366959190630143
       Electric: 2.461836046798032
            LPG: 2.781349448156026


In [130]:
#BODY TYPE __________________

body_type = df['body_type']
n_docs = len(df)
idf = {}


unique_body_type = body_type.drop_duplicates()

for body_type in unique_body_type:
    k = sum(body_type in row for row in df['body_type']) 
    idf[body_type] = np.log10(n_docs / (k + 1))  

for body_type, value in idf.items():
    print(f'{body_type:>15}: {value:>10}')

df['id_body_type'] = df['body_type'].map(idf)

df = df.astype({
    'id_body_type': float,
})

      Hatchback: 0.37473522552908584
            SUV: 0.5977701677077951
          Sedan: 0.562038335401452
            BMW: 3.842047288509638
            MUV: 1.3642028121708796
         Maruti: 2.9389573015176946
       Minivans: 2.435507108075683
  Mercedes-Benz: 3.2979792441593623
           Cars: 3.2979792441593623
          Coupe: 3.143077284173619
         Jaguar: 3.6659560294539566
          Wagon: 3.5410172928456567
         Datsun: 3.842047288509638
         Pickup: 3.143077284173619
           Tata: 3.364926033789976
          Honda: 3.842047288509638
   Convertibles: 3.6659560294539566
        Hyundai: 3.842047288509638
       Mahindra: 3.6659560294539566
          Volvo: 3.842047288509638
         Toyota: 3.842047288509638
          Isuzu: 3.842047288509638
          Skoda: 3.842047288509638
      Chevrolet: 3.842047288509638


In [131]:
# CITY __________________________________________________

city= df['city']
n_docs = len(df)
idf = {}


unique_cities = city.drop_duplicates()

for city in unique_cities:
    k = sum(city in row for row in df['city'])  
    idf[city] = np.log10(n_docs / (k + 1))  

for city, value in idf.items():
    print(f'{city:>15}: {value:>10}')


df['id_city'] = df['city'].map(idf)
df = df.astype({
    'id_city': float,
})

           Pune: 1.0911533681275127
         Jaipur: 1.2893790723974448
         Mumbai: 0.9167351970099885
        Kolkata: 1.1722656733011014
          Delhi: 0.7606999807055055
      Bangalore: 0.8696130115522729
      Hyderabad: 1.0334987372692326
        Chennai: 1.1310841695143623
        Gurgaon: 1.220871006734603
     Chandigarh: 1.6040011853808425
      Ahmedabad: 1.1181813240661342
        Lucknow: 1.5086000140128877
           Agra: 1.8902242531937261


In [132]:
print(df.dtypes)


full_name                object
registered_year          object
engine_capacity         float64
insurance                object
transmission_type        object
kms_driven              float64
owner_type              float64
fuel_type                object
max_power               float64
seats                   float64
mileage                 float64
body_type                object
city                     object
resale_price_Lakh       float64
id_nome                 float64
id_ano                  float64
id_insurance            float64
id_transmission_type    float64
id_fuel_type            float64
id_body_type            float64
id_city                 float64
dtype: object


In [133]:
df.columns

Index(['full_name', 'registered_year', 'engine_capacity', 'insurance',
       'transmission_type', 'kms_driven', 'owner_type', 'fuel_type',
       'max_power', 'seats', 'mileage', 'body_type', 'city',
       'resale_price_Lakh', 'id_nome', 'id_ano', 'id_insurance',
       'id_transmission_type', 'id_fuel_type', 'id_body_type', 'id_city'],
      dtype='object')

In [134]:
# Contar los valores nulos en cada columna
nulos_por_columna = df.isnull().sum()

print("Valores nulos por columna:")
print(nulos_por_columna)

Valores nulos por columna:
full_name               0
registered_year         0
engine_capacity         7
insurance               0
transmission_type       0
kms_driven              3
owner_type              0
fuel_type               0
max_power               0
seats                   0
mileage                 0
body_type               0
city                    0
resale_price_Lakh       0
id_nome                 0
id_ano                  0
id_insurance            0
id_transmission_type    0
id_fuel_type            0
id_body_type            0
id_city                 0
dtype: int64


In [135]:
# Método 1: Utilizando la propiedad shape
num_filas1 = df.shape[0]

# Método 2: Utilizando la función len()
num_filas2 = len(df)

print(f"Número de filas (Método 1): {num_filas1}")
print(f"Número de filas (Método 2): {num_filas2}")

Número de filas (Método 1): 13902
Número de filas (Método 2): 13902


In [136]:
df.fillna(0, inplace=True)

In [137]:
# Exportar colunas selecionadas para um novo DataFrame
colunas_selecionadas = ['id_nome', 'id_ano', 'engine_capacity', 'id_insurance', 'id_transmission_type','kms_driven', 'owner_type','id_fuel_type', 'max_power', 'seats', 'mileage', 'id_body_type', 'id_city','resale_price_Lakh']

colunas_selecionadas = ['id_nome', 'id_ano', 'engine_capacity', 'id_insurance', 'id_transmission_type', 'kms_driven', 'owner_type', 'id_fuel_type', 'max_power', 'seats', 'mileage', 'id_body_type', 'id_city', 'resale_price_Lakh']

# Verifica nulos nas colunas selecionadas
colunas_com_nulos = df[colunas_selecionadas].columns[df[colunas_selecionadas].isnull().any()].tolist()

if colunas_com_nulos:
    print("Colunas com valores nulos:", colunas_com_nulos)
    # Elimina linhas com valores nulos nessas colunas
    df.dropna(subset=colunas_com_nulos, inplace=True)
    print("Linhas com valores nulos eliminadas.")
else:
    print("Não há valores nulos nas colunas selecionadas.")

df_selecionado = df[colunas_selecionadas]
df_selecionado.to_csv('treino.csv', index=False)

Não há valores nulos nas colunas selecionadas.


In [138]:
# Método 1: Utilizando la propiedad shape
num_filas1 = df_selecionado.shape[0]

# Método 2: Utilizando la función len()
num_filas2 = len(df)

print(f"Número de filas (Método 1): {num_filas1}")
print(f"Número de filas (Método 2): {num_filas2}")

Número de filas (Método 1): 13902
Número de filas (Método 2): 13902


In [139]:
#Export
#Not the target
colunas_selecionadas = ['id_nome', 'id_ano','resale_price_Lakh']


if set(colunas_selecionadas).issubset(df.columns):
    df_selecionado = df[colunas_selecionadas]
    df_selecionado.to_csv('CenarioOld.csv', index=False)